# ** 실행 방법 **
```
uv pip install -e .
```

# 1. 워드 클라우드

In [ ]:
from dotenv import load_dotenv

load_dotenv()

stopwords = [
    "진짜",
    "정말",
    "영상",
    "채널",
    "구독",
    "좋아요",
    "댓글",
    "보고",
    "수",
    "거",
    "나",
    "그럼",
    "그런",
    "근데",
    "그런데",
    "이런",
    "저런",
    "것",
    "수",
    "등",
    "좀",
    "진짜",
    "ㅋㅋ",
    "ㅋㅋㅋ",
    "ㅋㅋㅋㅋ",
    "ㅎㅎ",
    "ㅎㅎㅎ",
    "아냐",
    "아니",
    "뭐",
    "왜",
    "하고",
    "하는",
    "하다",
    "됐다",
    "되다",
    "있다",
    "없다",
    "때",
    "때문",
    "이유",
    "전주",
    "익산",  # <- 특정 단어가 너무 흔해서 규칙을 망치면 넣어도 됨(선택)
]

In [ ]:
import pandas as pd

df = pd.read_csv("../../결과물/시각화/comments.csv")
texts = df["texts"].tolist()
print(texts)


In [ ]:
from lib.youtube import collect_all_comments

rows, texts = collect_all_comments("FVFRVGSvkq0")

print(len(rows), len(texts))
print(rows[:100], texts[:100])

In [ ]:
import pandas as pd

csv = pd.DataFrame({"texts": texts})
csv.to_csv("./시각화_결과물/comments.csv")

In [ ]:
from lib.visualization import word_cloud

word_cloud(texts, stopwords=stopwords, mask_img_path="../../images/circle.png")

# 2. 장바구니 분석

In [ ]:
import re


def clean_ko(s: str) -> str:
    s = s.lower()
    s = re.sub(r"[^0-9a-zA-Z가-힣\s]", " ", s)  # 특수문자 제거
    s = re.sub(r"\s+", " ", s).strip()
    return s


# 최소한의 불용어(필요하면 계속 추가)
STOP = {
    "그럼",
    "그런",
    "근데",
    "그런데",
    "이런",
    "저런",
    "것",
    "수",
    "등",
    "좀",
    "진짜",
    "ㅋㅋ",
    "ㅋㅋㅋ",
    "ㅋㅋㅋㅋ",
    "ㅎㅎ",
    "ㅎㅎㅎ",
    "아냐",
    "아니",
    "뭐",
    "왜",
    "하고",
    "하는",
    "하다",
    "됐다",
    "되다",
    "있다",
    "없다",
    "때",
    "때문",
    "이유",
    "전주",
    "익산",  # <- 특정 단어가 너무 흔해서 규칙을 망치면 넣어도 됨(선택)
}


def text_to_items(text: str, min_len: int = 2) -> list[str]:
    t = clean_ko(text)
    toks = [w for w in t.split() if len(w) >= min_len]
    toks = [w for w in toks if w not in STOP]
    return toks


items_list = [text_to_items(t) for t in texts]
items_list[:3]


In [ ]:
from kiwipiepy import Kiwi

kiwi = Kiwi()

# STOP_NOUN = {"전북","전주","익산","군산","완주","김제"}  # 너무 흔하면 제외(선택)
STOP_NOUN = []


def nouns_kiwi(text: str) -> list[str]:
    text = clean_ko(text)
    tokens = kiwi.tokenize(text)
    nouns = [t.form for t in tokens if t.tag.startswith("NN")]  # 명사류
    nouns = [n for n in nouns if len(n) >= 2 and n not in STOP_NOUN]
    return nouns


items_list = [nouns_kiwi(t) for t in texts]
df_items = pd.DataFrame({"items": items_list})


In [ ]:
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_arr = te.fit(items_list).transform(items_list)
te_arr


In [ ]:
import pandas as pd

# 문장에 각각의 단어들이 있는지 없는지를 True / False
df = pd.DataFrame(te_arr, columns=te.columns_)


In [ ]:
df.sum()

In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules

# support는 전체 문장 중에 itemset이 등장한 문장의 비율
# min_support : support가 min_support 이상인것만 보여주세요
# max_len : max_len개의 조합까지 보여주세요
frequent_itemsets = apriori(df, min_support=0.02, use_colnames=True, max_len=2)
frequent_itemsets

In [ ]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)
rules[["antecedents", "consequents", "support", "confidence", "lift"]]

In [ ]:
pivot_data = rules.head(20).pivot_table(
    index="antecedents",  # 행
    columns="consequents",  # 열
    values="lift",  # 기준
    fill_value=0,  # 매칭되지 않는 것은 이걸로 채워라.
)


In [ ]:
from lib.analysis import market_basket_ko

result = market_basket_ko(
    texts,
    min_support=0.02,
    max_len=2,
    top_k=20,
    stop_noun=["전북", "전주", "익산", "군산", "완주", "김제"],  # 필요하면
)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib  # noqa: F401


plt.figure(figsize=(10, 10))
sns.heatmap(
    result["pivot_data"],
    annot=True,
    cmap="YlGnBu",
    fmt=".2f",
    linewidths=0.3,
    square=True,
)
plt.title("연관성 분석 시각화(Lift 기준)")
plt.xlabel("Consequents")
plt.ylabel("Antecedents")
plt.show()

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import koreanize_matplotlib  # noqa: F401, F811

rules = result["rules"]
my_rules = rules.sort_values(by=["lift"], ascending=False).head(50)

# 1. 그래프 생성
G = nx.Graph()

# 2. 엣지 추가
for _, col_data in my_rules.iterrows():
    # 1) 단어 추출
    print(f"[BEFORE] {col_data['antecedents']}, {col_data['consequents']}")
    antecedent = ",".join(col_data["antecedents"])
    consequent = ",".join(col_data["consequents"])
    print(f"[AFTER] {antecedent}, {consequent}")
    print("=" * 100)

    # 2) 지표 추출
    weight = col_data["lift"]

    # 3) 그래프에 정보 추가
    G.add_edge(antecedent, consequent, weight=weight)

G.edges()

In [ ]:
# 3. 노드 배치
# position = nx.kamada_kawai_layout(G, scale=0.5)
# k를 조절하면 노드간 거리를 조절할 수 있습니다.
position = nx.spring_layout(G, k=0.9, seed=15)

# 4. 가중치 추출
scale = 0.3  ## 선의 굵기가 너무 굵다면 사이즈를 줄일 수 있습니다.
edge_weights = [G[u][v]["weight"] * scale for u, v in G.edges()]
print(edge_weights)

# 5. 그리기
plt.figure(figsize=(10, 10))
nx.draw_networkx_nodes(G, position, node_color="lightblue", node_size=1000)
nx.draw_networkx_edges(G, position, edge_color="gray", width=edge_weights)
nx.draw_networkx_labels(G, position, font_size=10, font_family="Malgun Gothic")
plt.title("단어 간 연관규칙 기반 네트워크 그래프(Lift 기준)")
plt.axis("off")
plt.show()